In [3]:
import torch
import torch.optim as optim
import joblib
import optuna
from model.model_bce_mse import MultiDecoderCondVAE, integrated_loss_fn

# 1. 환경 및 데이터 준비
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 데이터 로더 및 스케일러 로드
x_scaler = joblib.load('./torch/x_scaler.pkl')
train_loader = torch.load('torch/train_loader.pt', weights_only=False)
val_loader = torch.load('torch/val_loader.pt', weights_only=False)

# 입력 차원 자동 추출 (첫 번째 배치를 통해 확인)
x_sample, c_sample = next(iter(train_loader))
x_dim = x_sample.shape[1]
c_dim = c_sample.shape[1]
z1_dim = 32  # 기존 코드의 기본값 활용

def objective(trial):
    # 2. 튜닝할 하이퍼파라미터 제안
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    z_dim = trial.suggest_int("z_dim", 4, 32)
    alpha = trial.suggest_float("alpha", 0.1, 5.0)
    beta = trial.suggest_float("beta", 0.001, 0.1) # KL 가중치는 보통 작게 시작
    gamma = trial.suggest_float("gamma", 0.1, 10.0)
    
    # 3. 모델 및 옵티마이저 선언
    model = MultiDecoderCondVAE(x_dim, c_dim, z_dim, z1_dim).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    # 각 trial당 학습할 에포크 수 (성능 확인을 위해 최소 20~50회 권장)
    epochs = 50 
    
    for epoch in range(epochs):
        # --- Training Loop ---
        model.train()
        for x, c in train_loader:
            x, c = x.to(device), c.to(device)
            optimizer.zero_grad()
            
            mask_logits, prob_mask, mask_out, recon_numeric, z_mu, z_logvar = model(x, c)
            
            # integrated_loss_fn에 제안된 가중치 적용
            loss_dict = integrated_loss_fn(
                mask_logits, recon_numeric, x, z_mu, z_logvar, 
                alpha=alpha, beta=beta, gamma=gamma
            )
            
            loss_dict['loss'].backward()
            optimizer.step()
        
        # --- Validation Loop ---
        model.eval()
        v_total_loss = 0
        with torch.no_grad():
            for v_x, v_c in val_loader:
                v_x, v_c = v_x.to(device), v_c.to(device)
                v_mask_logits, _, _, v_recon_numeric, v_z_mu, v_z_logvar = model(v_x, v_c)
                
                v_loss_dict = integrated_loss_fn(
                    v_mask_logits, v_recon_numeric, v_x, v_z_mu, v_z_logvar,
                    alpha=alpha, beta=beta, gamma=gamma
                )
                v_total_loss += v_loss_dict['loss'].item()
        
        avg_val_loss = v_total_loss / len(val_loader)
        
        # Pruning: 성능이 개선되지 않는 trial은 조기 종료하여 시간 절약
        trial.report(avg_val_loss, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return avg_val_loss

# 4. 최적화 실행
# n_trials: 총 시도 횟수 (예: 30)
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30)

# 5. 최적화 결과 확인 및 모델 재학습 활용
print("-" * 30)
print("Best hyperparameters:", study.best_params)
print("Best validation loss:", study.best_value)

# 6. (선택) 파라미터 중요도 시각화
# optuna.visualization.plot_param_importances(study).show()

[I 2026-01-23 20:04:19,968] A new study created in memory with name: no-name-b133c32c-23ad-4253-8bab-8c8923c7da27
[I 2026-01-23 20:04:38,784] Trial 0 finished with value: 6.623849216260408 and parameters: {'lr': 0.00012202850206809438, 'z_dim': 28, 'alpha': 1.649311977256356, 'beta': 0.06591208039212863, 'gamma': 6.305661915211629}. Best is trial 0 with value: 6.623849216260408.
[I 2026-01-23 20:04:52,547] Trial 1 finished with value: 3.342386245727539 and parameters: {'lr': 0.0003226516839747432, 'z_dim': 8, 'alpha': 2.6752288784387384, 'beta': 0.0685914693164728, 'gamma': 0.8448321864548642}. Best is trial 1 with value: 3.342386245727539.
[I 2026-01-23 20:05:06,545] Trial 2 finished with value: 3.2732495508695902 and parameters: {'lr': 0.0008547619347206726, 'z_dim': 27, 'alpha': 1.7271285430701393, 'beta': 0.009528470783378606, 'gamma': 3.3709070530985104}. Best is trial 2 with value: 3.2732495508695902.
[I 2026-01-23 20:05:20,527] Trial 3 finished with value: 4.603539303729408 and 

------------------------------
Best hyperparameters: {'lr': 0.006766126204772056, 'z_dim': 21, 'alpha': 0.11166089758490058, 'beta': 0.03679915945155675, 'gamma': 0.14792382594314013}
Best validation loss: 0.1445702988850443
